### Step 4

In [ ]:
# This allows us to show the full screen width

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
BLmod = pd.read_excel("Assignment Data/Week 3/Bundesliga prediction model (Assignment).xlsx")
BLmod

In [ ]:
# define the predicted result based on the betting odds 

BLmod['B365res']= np.where((BLmod['B365H']<BLmod['B365D']) & (BLmod['B365H']<BLmod['B365A']),'H',\
                        np.where((BLmod['B365D']<BLmod['B365H']) & (BLmod['B365D']<BLmod['B365A']),'D',\
                        np.where((BLmod['B365A']<BLmod['B365H']) & (BLmod['B365A']<BLmod['B365D']),'A',"")))

In [ ]:
# define the probabilities associated with bookmaker odds

BLmod['B365probH'] = 1/BLmod['B365H']/(1/BLmod['B365H']+1/BLmod['B365D']+1/BLmod['B365A'])
BLmod['B365probD'] = 1/BLmod['B365D']/(1/BLmod['B365H']+1/BLmod['B365D']+1/BLmod['B365A'])
BLmod['B365probA'] = 1/BLmod['B365A']/(1/BLmod['B365H']+1/BLmod['B365D']+1/BLmod['B365A'])

# we now create a winvalue (for the home team, 2 = win, 1 = draw, 0 = loss) which we will need for the order logit regression.

BLmod['winvalue'] = np.where(BLmod['FTR'] == 'H', 2 ,(np.where(BLmod['FTR'] == 'D', 1, 0)))

In [ ]:
# crosstab showing how often the bookmaker odds correctly predict the result

pd.crosstab(BLmod['FTR'], BLmod['B365res'],dropna= True)

In [ ]:
# create the log of salary ratios 
BLmod['lhTMratio'] = np.log(BLmod['Tmhome']/BLmod['Tmaway'])

In [ ]:
BLtrain = BLmod[:224]
BLtrain

In [ ]:
from bevel.linear_ordinal_regression import OrderedLogit
ol = OrderedLogit()

In [ ]:
ol.fit(BLtrain['lhTMratio'], BLtrain['winvalue'])
ol.print_summary()

In [ ]:
#%% To get the coefficients and the intercepts
print(f'beta = {ol.coef_[0]:.4f}')
print(f'interceptAD = {ol.coef_[1]:.4f}')
print(f'interceptDH = {ol.coef_[2]:.4f}')

In [ ]:
# Predicted probabilities

BLmod['predA'] = 1/(1+np.exp(-(ol.coef_[1]-ol.coef_[0]*BLmod['lhTMratio'])))
BLmod['predD'] = 1/(1+np.exp(-(ol.coef_[2]-ol.coef_[0]*BLmod['lhTMratio']))) - BLmod['predA']
BLmod['predH'] = 1 - BLmod['predA'] - BLmod['predD']

pd.set_option('display.max_columns', 50)
BLmod

In [ ]:
# Result prediction

BLmod['Maxprob'] =BLmod[['predA','predD','predH']].max(axis=1)
BLmod['logitpred']=np.where(BLmod['Maxprob']==BLmod['predA'],'A',\
                               np.where(BLmod['Maxprob']==BLmod['predD'],'D','H'))
BLmod['logittrue']= np.where(BLmod['logitpred'] == BLmod['FTR'],1,0)
BLmod

In [ ]:
BLfore = BLmod[224:306]

In [ ]:
# Outcome value for calculating Brier Score

BLfore['Houtcome']= np.where(BLfore['FTR']=='H',1,0)
BLfore['Doutcome']= np.where(BLfore['FTR']=='D',1,0)
BLfore['Aoutcome']= np.where(BLfore['FTR']=='A',1,0)
BLfore

In [ ]:
BLfore['B365true']=np.where(BLfore['FTR']==BLfore['B365res'],1,0)

In [ ]:
BLfore['B365true'].mean()

In [ ]:
BLfore['logittrue'].mean()

In [ ]:
BLfore['BrierB365'] = (BLfore['B365probH'] - BLfore['Houtcome'])**2 +(BLfore['B365probD'] - BLfore['Doutcome'])**2 +(BLfore['B365probA'] - BLfore['Aoutcome'])**2
BLfore['Brierlogit'] = (BLfore['predH'] - BLfore['Houtcome'])**2 +(BLfore['predD'] - BLfore['Doutcome'])**2 + (BLfore['predA'] - BLfore['Aoutcome'])**2


In [ ]:
BLfore['BrierB365'].mean()

In [ ]:
BLfore['Brierlogit'].mean()